In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import time
from copy import deepcopy

In [ ]:
# Check if CUDA is available
if torch.cuda.is_available():
    device = torch.device("cuda")
    torch.cuda.empty_cache()
    print("CUDA is available. Using GPU.")
else:
    device = torch.device("cpu")
    print("CUDA is not available. Using CPU.")

In [3]:
# Custom transform to convert grayscale images to RGB
class GrayscaleToRGB:
    def __call__(self, img):
        return img.convert("RGB")

In [4]:
criterion = nn.CrossEntropyLoss()

In [5]:
# FGSM attack implementation
def fgsm_attack(model, data, target, epsilon):
    data.requires_grad = True
    output = model(data)
    loss = criterion(output, target)
    model.zero_grad()
    loss.backward()
    data_grad = data.grad.data
    perturbed_data = data + epsilon * data_grad.sign()
    return perturbed_data

In [6]:
# Define PGD attack
def pgd_attack(model, data, target, epsilon, alpha, num_iter):
    perturbed_data = data.clone().detach().requires_grad_(True).to(data.device)
    for _ in range(num_iter):
        output = model(perturbed_data)
        loss = criterion(output, target)
        model.zero_grad()
        loss.backward()
        perturbed_data = perturbed_data + alpha * perturbed_data.grad.sign()
        perturbed_data = torch.clamp(perturbed_data, data - epsilon, data + epsilon)
        perturbed_data = torch.clamp(perturbed_data, 0, 1).detach().requires_grad_(True)
    return perturbed_data

In [7]:
# Define DeepFool attack
def deepfool_attack(model, data, num_classes=10, overshoot=0.02, max_iter=50):
    data = data.clone().detach().requires_grad_(True).to(data.device)
    perturbed_data = data.clone().detach().requires_grad_(True).to(data.device)
    output = model(perturbed_data)
    _, pred = torch.max(output.data, 1)
    for _ in range(max_iter):
        output = model(perturbed_data)
        loss = criterion(output, pred)
        model.zero_grad()
        loss.backward()
        grad = perturbed_data.grad.data
        perturbed_data = perturbed_data + overshoot * grad.sign()
        perturbed_data = torch.clamp(perturbed_data, 0, 1).detach().requires_grad_(True)
        output = model(perturbed_data)
        _, new_pred = torch.max(output.data, 1)
        if not torch.equal(new_pred, pred):
            break
    return perturbed_data

In [8]:
# Define CW attack
def cw_attack(model, data, target, c=1e-4, kappa=0, max_iter=1000, learning_rate=0.01):
    data = data.clone().detach().requires_grad_(True).to(data.device)
    target = target.to(data.device)
    optimizer = optim.Adam([data], lr=learning_rate)
    for i in range(max_iter):
        output = model(data)
        loss = criterion(output, target)
        model.zero_grad()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            data = torch.clamp(data, 0, 1)
    return data

In [9]:
# Define a simple CNN model
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3)
        self.fc1 = nn.Linear(64 * 12 * 12 * 4, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [10]:
# Define LeNet-5 model
class LeNet5(nn.Module):
    def __init__(self):
        super(LeNet5, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, kernel_size=5)
        self.conv2 = nn.Conv2d(6, 16, kernel_size=5)
        self.fc1 = nn.Linear(16 * 4 * 4, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.max_pool2d(x, 2)
        x = torch.relu(self.conv2(x))
        x = torch.max_pool2d(x, 2)
        x = torch.flatten(x, 1)
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [11]:
# Define VGG16 model
class VGG16(nn.Module):
    def __init__(self, num_classes=10):
        super(VGG16, self).__init__()
        self.features = models.vgg16(pretrained=True).features
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )

    def forward(self, x):
        x = self.features(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

In [12]:
# Define ResNet18 model
class ResNet18(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet18, self).__init__()
        self.model = models.resnet18(pretrained=True)
        self.model.fc = nn.Linear(self.model.fc.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [13]:
# Define DenseNet121 model
class DenseNet121(nn.Module):
    def __init__(self, num_classes=10):
        super(DenseNet121, self).__init__()
        self.model = models.densenet121(pretrained=True)
        self.model.classifier = nn.Linear(self.model.classifier.in_features, num_classes)

    def forward(self, x):
        return self.model(x)

In [14]:
# Define LSTM model
class LSTMModel(nn.Module):
    def __init__(self, input_size=28, hidden_size=128, num_layers=2, num_classes=10):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, num_classes)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.lstm(x, (h0, c0))
        out = self.fc(out[:, -1, :])
        return out

In [15]:
# Define Transformer model
class TransformerModel(nn.Module):
    def __init__(self, input_size=28, num_classes=10, d_model=128, nhead=8, num_layers=2):
        super(TransformerModel, self).__init__()
        self.embedding = nn.Linear(input_size, d_model)
        self.pos_encoder = nn.Parameter(torch.zeros(1, 28, d_model))
        self.transformer_encoder = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead),
            num_layers=num_layers
        )
        self.fc = nn.Linear(d_model, num_classes)

    def forward(self, x):
        x = self.embedding(x) + self.pos_encoder
        x = x.permute(1, 0, 2)  # Transformer expects (seq_len, batch, d_model)
        x = self.transformer_encoder(x)
        x = x.mean(dim=0)  # Average over sequence length
        x = self.fc(x)
        return x

In [16]:
def train_model(model, interations, train_loader, optimizer, criterion, device, remove_channel_dim=False):
    model.train()
    for i in range(interations):
        for data, target in train_loader:
            data, target = data.to(device), target.to(device)
            if remove_channel_dim:
                data = data.squeeze(1)
            if isinstance(model, LSTMModel) or isinstance(model, TransformerModel):
                data = data.view(data.size(0), -1, data.size(-1))
            optimizer.zero_grad()
            output = model(data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

In [17]:
def load_mnist_dataset(transform_needed=False):
    transform = transforms.Compose([
        GrayscaleToRGB(),
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]) if transform_needed else transforms.ToTensor()
    train_dataset = datasets.MNIST(root="data", train=True, download=True, transform=transform)
    test_dataset = datasets.MNIST(root="data", train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    return train_loader, test_loader

In [18]:
def load_fashion_mnist_dataset(transform_needed=False):
    transform = transforms.Compose([
        GrayscaleToRGB(),
        transforms.Resize((32, 32)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ]) if transform_needed else transforms.ToTensor()
    train_dataset = datasets.FashionMNIST(root="data", train=True, download=True, transform=transform)
    test_dataset = datasets.FashionMNIST(root="data", train=False, download=True, transform=transform)
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)
    return train_loader, test_loader

In [19]:
def check_results_before_adversarial_attack(model, test_loader, device):
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            if isinstance(model, LSTMModel) or isinstance(model, TransformerModel):
                data = data.squeeze(1)  
                data = data.view(data.size(0), -1, data.size(-1)) 
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            total += target.size(0)
    accuracy = correct / total
    print(f'Accuracy before adversarial attack: {accuracy * 100:.2f}%')
    return accuracy

In [20]:
def check_results_after_adversarial_attack(model, test_loader, attack_fn, attack_params, device):
    if isinstance(model, LSTMModel) or isinstance(model, TransformerModel):
        model.train()
    else:
        model.eval()
    correct = 0
    total = 0
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        if isinstance(model, LSTMModel) or isinstance(model, TransformerModel):
            data = data.squeeze(1) 
            data = data.view(data.size(0), -1, data.size(-1))  
        perturbed_data = attack_fn(model, data, target, **attack_params)
        output = model(perturbed_data)
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()
        total += target.size(0)
        if isinstance(model, LSTMModel) or isinstance(model, TransformerModel):
            model.train()
    accuracy = correct / total
    print(f'Accuracy after adversarial attack: {accuracy * 100:.2f}%')
    return accuracy

In [ ]:
# Lista modeli do przetestowania
models = {
    "SimpleCNN": SimpleCNN(),
    "LeNet5": LeNet5(),
    "VGG16": VGG16(num_classes=10),
    "ResNet18": ResNet18(num_classes=10),
    "DenseNet121": DenseNet121(num_classes=10),
    "LSTMModel": LSTMModel(input_size=28, hidden_size=128, num_layers=2, num_classes=10),
    "TransformerModel": TransformerModel(input_size=28, num_classes=10, d_model=128, nhead=8, num_layers=2)
}

In [22]:
experimental_datasets = {
    "MNIST": load_mnist_dataset,
    "FashionMNIST": load_fashion_mnist_dataset
}

In [23]:
attacks = {
    "FGSM_1": (fgsm_attack, {"epsilon": 0.3}),
    "FGSM_2": (fgsm_attack, {"epsilon": 0.1}),
    "FGSM_3": (fgsm_attack, {"epsilon": 0.05}),
    "FGSM_4": (fgsm_attack, {"epsilon": 0.01}),
    "PGD_1": (pgd_attack, {"epsilon": 0.3, "alpha": 0.01, "num_iter": 50}),
    "PGD_2": (pgd_attack, {"epsilon": 0.1, "alpha": 0.01, "num_iter": 50}),
    "PGD_3": (pgd_attack, {"epsilon": 0.2, "alpha": 0.02, "num_iter": 100}),
    "PGD_4": (pgd_attack, {"epsilon": 0.05, "alpha": 0.005, "num_iter": 30}),
    "DeepFool_1": (deepfool_attack, {"overshoot": 0.02, "max_iter": 10000}),
    "DeepFool_2": (deepfool_attack, {"overshoot": 0.01, "max_iter": 5000}),
    "DeepFool_3": (deepfool_attack, {"overshoot": 0.03, "max_iter": 1000}),
    "DeepFool_4": (deepfool_attack, {"overshoot": 0.005, "max_iter": 200}),
    "CW_1": (cw_attack, {"c": 1e-3, "kappa": 0, "max_iter": 1000, "learning_rate": 0.01}),
    "CW_2": (cw_attack, {"c": 1e-2, "kappa": 0, "max_iter": 500, "learning_rate": 0.01}),
    "CW_3": (cw_attack, {"c": 1e-4, "kappa": 0, "max_iter": 2000, "learning_rate": 0.005}),
    "CW_4": (cw_attack, {"c": 1e-3, "kappa": 0.5, "max_iter": 1000, "learning_rate": 0.01})
}

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

for model_name, model in models.items():
    print(f"\nTesting model: {model_name}")
    model = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.CrossEntropyLoss()

    for dataset_name, load_dataset in experimental_datasets.items():
        print(f"\nTesting dataset: {dataset_name}")
        train_loader, test_loader = load_dataset(transform_needed=(model_name in ["VGG16", "ResNet18", "DenseNet121"]))

        start_time = time.time()
        train_model(model, 10, train_loader, optimizer, criterion, device, remove_channel_dim=(model_name in ["LSTMModel", "TransformerModel"]))
        training_time = time.time() - start_time

        accuracy_before = check_results_before_adversarial_attack(model, test_loader, device)
        print(f"Training time: {training_time:.2f} seconds")

        for attack_name, (attack_fn, attack_params) in attacks.items():
            print(f"\nTesting attack: {attack_name} with parameters: {attack_params}")

            model_copy = deepcopy(model)
            model_copy = model_copy.to(device)

            start_time = time.time()
            accuracy_after = check_results_after_adversarial_attack(model_copy, test_loader, attack_fn, attack_params, device)
            attack_time = time.time() - start_time
            print(f"Attack time: {attack_time:.2f} seconds")
            # print(f"Accuracy before attack: {accuracy_before * 100:.2f}%")
            # print(f"Accuracy after attack: {accuracy_after * 100:.2f}%")

            del model_copy, accuracy_after
            torch.cuda.empty_cache()

        del train_loader, test_loader, accuracy_before